In [1]:
%load_ext sql
%sql sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

In [2]:
%%sql
ATTACH 'CourseData.db' AS CDDB

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [3]:
%%sql
select * from CDDB.PROGRAMS LIMIT 1

 * sqlite:///CourseDataWarehouse.db
Done.


PID,program_code,program_name
1,AN,Asian Studies


In [204]:
%%sql 
DROP TABLE IF EXISTS PROGRAMS_DIM;
DROP TABLE IF EXISTS LOCATIONS_DIM;
DROP TABLE IF EXISTS TIMES_DIM;
DROP TABLE IF EXISTS FACULTY_DIM;
DROP TABLE IF EXISTS COURSE_CATALOGS_DIM;
DROP TABLE IF EXISTS COURSES_FACT;

CREATE TABLE PROGRAMS_DIM (
    PID INTEGER NOT NULL PRIMARY KEY,
    program_code TEXT NOT NULL,
    program_name TEXT NOT NULL
);


CREATE TABLE LOCATIONS_DIM (
    LID INTEGER PRIMARY KEY,
    location TEXT ,
    BuildingName TEXT

);


CREATE TABLE FACULTY_DIM (
    FID  INTEGER NOT NULL PRIMARY KEY,
    instructor_fname TEXT NOT NULL,
    instructor_lname TEXT NOT NULL
);


CREATE TABLE COURSE_CATALOGS_DIM (
    CID INTEGER PRIMARY KEY,
    Catalog_ID TEXT NOT NULL,
    Credits TEXT NOT NULL,
    Prereqs TEXT,
    Coreqs TEXT,
    Fees INTEGER,
    Description TEXT,
    Attributes TEXT
);



CREATE TABLE COURSES_FACT (
    CID INTEGER NULL ,
    CRN INTEGER NOT NULL,
    SEMESTER TEXT NOT NULL ,
    YEAR INTEGER NOT NULL,
    Title TEXT NOT NULL,
    SECTION TEXT NOT NULL,
    Act INTEGER,
    Cap INTEGER,
    Rem INTEGER, 
    timecodes TEXT,
    PID INTEGER NOT NULL,
    LID INTEGER NOT NULL,
    FID INTEGER NOT NULL,
    catalog_ID INTEGER  NULL,
    FOREIGN KEY (PID) REFERENCES PROGRAMS_DIM (PID),
    FOREIGN KEY (LID) REFERENCES LOCATIONS_DIM (LID),
    FOREIGN KEY (FID) REFERENCES FACULTY_DIM (FID),
    FOREIGN KEY (CID) REFERENCES COURSE_CATALOGS_DIM (CID)
);

 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [205]:
%%sql
SELECT * FROM  COURSES_FACT

 * sqlite:///CourseDataWarehouse.db
Done.


CID,CRN,SEMESTER,YEAR,Title,SECTION,Act,Cap,Rem,timecodes,PID,LID,FID,catalog_ID


In [206]:
%%sql 


INSERT INTO PROGRAMS_DIM (program_name, program_code)
SELECT DISTINCT program_name, program_code
FROM CDDB.PROGRAMS;


INSERT INTO LOCATIONS_DIM (location ,BuildingName )
SELECT DISTINCT location , BuildingName
FROM CDDB.LOCATION;


INSERT INTO FACULTY_DIM (instructor_fname , instructor_lname)
SELECT DISTINCT instructor_fname , instructor_lname
FROM CDDB.FACULTY;


INSERT INTO COURSE_CATALOGS_DIM (Catalog_ID, credits, Prereqs, Coreqs, Fees, Description, Attributes)
SELECT DISTINCT catalog_ID, credits, prereqs, coreqs, fees, description, attributes
FROM CDDB.COURSE;



INSERT INTO COURSES_FACT (CRN, SEMESTER , YEAR , TITLE, SECTION, ACT, CAP,REM, TIMECODES, PID, LID, FID, CID)
SELECT DISTINCT CDDB.CLASS_MEETINGS.CRN, CDDB.CLASS_MEETINGS.SEMESTER , CDDB.CLASS_MEETINGS.YEAR, CDDB.COURSE.course_title, CDDB.COURSE_OFFERING.SECTION, act, cap, rem, timecodes, PID, LOCATIONS_DIM.LID, FID,CDDB.COURSE.CID
FROM CDDB.COURSE_OFFERING
     JOIN CDDB.CLASS_MEETINGS USING  (COID)
    left JOIN LOCATIONS_DIM USING (LID)
    left JOIN CDDB.COURSE USING (CID)
    left JOIN PROGRAMS_DIM USING (PID)
    left JOIN FACULTY_DIM USING (FID)
    JOIN COURSE_CATALOGS_DIM USING (CID);
    
    

    
   

 * sqlite:///CourseDataWarehouse.db
83 rows affected.
207 rows affected.
1104 rows affected.
2220 rows affected.
2700 rows affected.


[]

In [207]:
%%sql
SELECT FID, instructor_lname,instructor_fname,COURSE_CATALOGS_DIM.Catalog_ID,program_name, Title ,sum(act), sum(cap), sum(rem) , YEAR,SEMESTER , COUNT(CRN) as COURSESTaught
FROM FACULTY_DIM
    JOIN COURSES_FACT USING (FID)
    JOIN COURSE_CATALOGS_DIM USING (CID)
    JOIN PROGRAMS_DIM USING (PID)
WHERE (instructor_lname like '%Huntley%')
GROUP BY title
ORDER BY COURSESTaught DESC


 * sqlite:///CourseDataWarehouse.db
Done.


FID,instructor_lname,instructor_fname,Catalog_ID,program_name,Title,sum(act),sum(cap),sum(rem),YEAR,SEMESTER,COURSESTaught
275,Huntley,Christopher L.,IS 0240,Information Systems,Systems Analysis and Logical Design,45,50,5,2017,Fall,2
275,Huntley,Christopher L.,IS 0260,Information Systems,Database Systems,24,25,1,2018,Spring,1
275,Huntley,Christopher L.,IS 0510,Information Systems,Databases for Business Analytics,18,20,2,2017,Fall,1
275,Huntley,Christopher L.,IS 0505,Information Systems,Python for Business Analytics,28,20,-8,2017,Fall,1
275,Huntley,Christopher L.,IS 0320,Information Systems,Systems Design and Implementation,20,25,5,2018,Spring,1


In [172]:
%%sql
SELECT DISTINCT COUNT(CRN), COUNT(CID), COUNT(CRN)
FROM COURSES_FACT;

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(CRN),COUNT(CID),COUNT(CRN)_1
2700,2700,2700


In [208]:
%%sql

SELECT CRN,COURSE_CATALOGS_DIM.Catalog_ID, program_name,  instructor_lname,instructor_fname, YEAR,location ,BuildingName
FROM COURSES_FACT
    JOIN PROGRAMS_DIM USING (PID)
    JOIN COURSE_CATALOGS_DIM USING (CID)
    JOIN FACULTY_DIM USING (FID)
    JOIN LOCATIONS_DIM USING (LID)
WHERE program_name like 'iNFORMATION%' AND YEAR = 2018
GROUP BY CRN
ORDER BY program_name DESC

    

 * sqlite:///CourseDataWarehouse.db
Done.


CRN,Catalog_ID,program_name,instructor_lname,instructor_fname,YEAR,location,BuildingName
11834,IS 0500,Information Systems,Ozcelik,Yasin,2018,DSB 106,Dolan School of Business
34147,IS 0100,Information Systems,Vinekar,Vishnu V.,2018,DSB 106,Dolan School of Business
34148,IS 0100,Information Systems,Vinekar,Vishnu V.,2018,DSB 106,Dolan School of Business
34149,IS 0100,Information Systems,Ozcelik,Yasin,2018,DSB 110B,Dolan School of Business
34150,IS 0100,Information Systems,Ozcelik,Yasin,2018,DSB 110B,Dolan School of Business
34151,IS 0100,Information Systems,Wang,Kanlun,2018,DSB 110A,Dolan School of Business
34641,IS 0100,Information Systems,Tao,Jie,2018,DSB 114,Dolan School of Business
34642,IS 0135,Information Systems,Vinekar,Vishnu V.,2018,DSB 106,Dolan School of Business
34643,IS 0210,Information Systems,Lee,Patrick S.,2018,DSB 114,Dolan School of Business
34644,IS 0240,Information Systems,Huntley,Christopher L.,2018,DSB 112,Dolan School of Business


In [173]:
%%sql
vacuum; 

 * sqlite:///CourseDataWarehouse.db
Done.


[]